In [9]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

In [11]:
# Verifica se a GPU está disponível
gpu_available = torch.cuda.is_available()

if gpu_available:
    # Obtém o número de GPUs disponíveis
    num_gpus = torch.cuda.device_count()

    # Obtém o ID da GPU atualmente em uso
    current_gpu = torch.cuda.current_device()

    print(f"Disponibilidade de GPU: {gpu_available}")
    print(f"Número de GPUs disponíveis: {num_gpus}")
    print(f"ID da GPU atualmente em uso: {current_gpu}")
else:
    print("GPU não está disponível. Utilizando CPU.")

Disponibilidade de GPU: True
Número de GPUs disponíveis: 1
ID da GPU atualmente em uso: 0


In [15]:
# Carregar dados
data = pd.read_csv('../TCC/datasets/forecast_dap.csv')
data['Timestamp'] = pd.to_datetime(data['Timestamp']).dt.strftime('%Y-%m-%d %H:%M')
data.set_index('Timestamp', inplace=True)

# Escolher a coluna alvo
target_column = 'Day Ahead Price'
data = data[[target_column]]

# Normalizar os dados
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data)

# Definir janela de tempo e criar sequências
window_size = 24  # Escolher o tamanho da janela para previsão
train_data = data_normalized[:-window_size]  # Dados de treino
test_data = data_normalized[-window_size:]  # Últimos 24 pontos para teste

# Converter para tensores
def create_sequences(data, window_size):
    sequences = []
    for i in range(len(data) - window_size):
        sequence = data[i:i+window_size]
        sequences.append(sequence)
    return np.array(sequences)

train_sequences = create_sequences(train_data, window_size)
test_sequences = create_sequences(test_data, window_size)

# Converter para tensores do PyTorch
train_sequences_tensor = torch.from_numpy(train_sequences).float()
test_sequences_tensor = torch.from_numpy(test_sequences).float()

# DataLoader
train_dataset = TensorDataset(train_sequences_tensor)
train_loader = DataLoader(train_dataset, batch_size=4096, shuffle=True)

# Definir modelo LSTM
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

# Inicializar o modelo
model = LSTM()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Verifique se a GPU está disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Mova o modelo e os tensores para a GPU, se disponível
model.to(device)
train_sequences_tensor = train_sequences_tensor.to(device)
test_sequences_tensor = test_sequences_tensor.to(device)

# Treinar o modelo
epochs = 100
print_every = 10  # Imprimir a perda a cada 10 iterações

for epoch in range(epochs):
    # Usar tqdm para criar a barra de progresso
    loop = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
    for i, seq in loop:
        optimizer.zero_grad()
        model.hidden_cell = (
            torch.zeros(1, 1, model.hidden_layer_size).to(device),
            torch.zeros(1, 1, model.hidden_layer_size).to(device)
        )

        seq = seq[0].view(-1, 1, 1).to(device)
        y_pred = model(seq)

        y_pred = y_pred.view(-1)

        loss = criterion(y_pred, seq[-1].view(-1))
        loss.backward()
        optimizer.step()

        if i % print_every == 0:
            loop.set_description(f'Epoch [{epoch+1}/{epochs}]')
            loop.set_postfix(loss=loss.item())  # Atualizar a perda na barra de progresso

# Fazer previsões
future = 24
model.eval()
with torch.no_grad():
    test_seq = test_sequences_tensor[:1].to(device)
    preds = []
    for _ in range(future):
        test_seq = test_seq.view(-1, 1, 1)

        y_test_pred = model(test_seq)
        preds.append(y_test_pred.item())

        new_seq = test_seq.cpu().numpy().flatten()
        new_seq = np.append(new_seq, y_test_pred.cpu())
        new_seq = new_seq[1:]
        test_seq = torch.as_tensor(new_seq).view(1, -1, 1).float().to(device)

# Inverter a normalização
predicted_values = scaler.inverse_transform(np.array(preds).reshape(-1, 1))
print("Previsões para os próximos 24 pontos:", predicted_values.flatten())

Device: cuda


KeyboardInterrupt: 